In [74]:
from random import randint
#@title Librerias
import numpy as np
from numpy import random
import numpy.linalg as la
import matplotlib.pyplot as plt
from scipy.integrate import quad, fixed_quad, trapezoid, simpson, nquad
from sympy import symbols, diff, integrate, lambdify

# Parcial II - Métodos Computacionales (Integrales y Matrices)
**2024-2**

---

*Nombre:*

*Identificación:*

---
* Sea honest@
* El uso de herramientas de IA no está permitido
* Puede consultar la documentación de las librerias.
* Sea ordendado dando la solución a cada punto.
* *Asegurese que el notebook corra completamente*
* Suba su notebook al GitHub (en su carpeta) y recuerde hacer pull-request

# 1

**(20 puntos)**
Considera la siguiente integral:

$$
I = \int_0^1 \int_{x^2}^{x+1} \int_{y^2}^{y+2} \int_{z^2}^{z+3} (x+y+z+w) \, dw \, dz \, dy \, dx
$$

Encuentre la solución númerica usando unicamente `quad`. Encuentre el error relativo, para esto emplee el siguiente código que usa `nquad`:


```python
# Definir el integrando
integrand = lambda w, z, y, x: x + y + z + w

# Definir los límites usando funciones lambda
w_limits = lambda z, y, x: [z**2, z+3]
z_limits = lambda y, x: [y**2, y+2]
y_limits = lambda x: [x**2, x+1]
x_limits = lambda: [0, 1]

# Calcular la integral usando nquad
result, error = nquad(
    integrand,
    [w_limits, z_limits, y_limits, x_limits]
)

print(f"Resultado de la integral: {result}")
print(f"Error estimado: {error}")
```



In [130]:
def integrando(w, z, y, x):
    return x + y + z + w


w_g = lambda z: z**2
w_h = lambda z: z + 3

z_e = lambda y: y**2
z_f = lambda y: y + 2

y_c = lambda x: x**2
y_d = lambda x: x + 1


In [131]:

def Integral_w(z, y, x):
    return quad(lambda w: integrando(w, z, y, x), w_g(z), w_h(z))[0]


def Integral_z(y, x):
    return quad(lambda z: Integral_w(z, y, x), z_e(y), z_f(y))[0]


def Integral_y(x):
    return quad(lambda y: Integral_z(y, x), y_c(x), y_d(x))[0]


resultado, error = quad(lambda x: Integral_y(x), 0, 1)


In [132]:
resultado

-0.6500971589021813

In [134]:
result, error = nquad(
    integrand,
    [w_limits, z_limits, y_limits, x_limits]
)

In [136]:
#error relativo
error_r=abs(result-resultado)/result
error_r

-0.0

# 2

**(40 puntos)**
Un lazo de corriente de radio $a$  transporta una corriente $I$ en el punto P que está a una distancia $r$ del centro del lazo con coordenadas esféricas (r, θ, φ). Resuelve para la componente φ del potencial vectorial en el punto $P$ en términos de integrales elípticas:

\begin{equation}
A_{\phi}(r, \theta)=\frac{\mu_0}{4\pi}\frac{4Ia}{\sqrt{a^2+r^2+2ar\sin \theta}}\left(\frac{(2-k^2)K(k)-2E(k)}{k^2} \right)
\end{equation}

donde

\begin{equation}
K(k)=\int_{0}^{\pi/2} \frac{\mathrm{d\phi}}{\sqrt{1-k^2\sin^2\phi}}
\end{equation}

\begin{equation}
E(k)=\int_{0}^{\pi/2} \mathrm{d\phi} \sqrt{1-k^2\sin^2\phi}
\end{equation}

\begin{equation}
k^2=\frac{4ar\sin\theta}{a^2+r^2+2ar\sin\theta}
\end{equation}

Aquí, $K(k)$ es una integral elíptica completa del primer tipo y $E(k)$ es una integral elíptica completa del segundo tipo.


Para
* $a$ = 1,
* $I$ = 3,
* $μ0/4π$ = 1


Usando funciones explicitas explique claramente las entradas, salidas y procedimientos dentro de las mismas. Recuerde para resolver las integrales usar `quad` y grafique para:

*a)* $A_φ(r = 1.1, θ)$ vs. $0\leq θ\leq \pi$

*b)* $A_φ(r, θ = π∕3)$ vs. $0\leq r \leq 10$

Tenga cuidado con la convergencia de su integral



In [104]:
#definimos:
a=1
I=3
u=1

In [108]:
k=np.linspace(-5,5,100)

In [147]:
def k_f2(r,fi):
    return (4*a*r*np.sin(fi))/(a**2+r**2+2*a*r*np.sin(fi))

In [143]:
def E_int(k):
 integrand_k = lambda fi: np.sqrt(1-k**2*np.sin(fi)**2)
 return quad(integrand_k,0,np.pi/2)[0]


In [ ]:
def E(k):
    integrand = lambda fi: np.sqrt(1 - k**2 * np.sin(fi)**2)
    return quad(integrand, 0, np.pi/2)[0]

In [149]:
integral_k=quad(K_f2(k),0,np.pi/2)

NameError: name 'K_f2' is not defined

# 3

(**40 puntos**) El **método de la potencia inversa** es una técnica iterativa utilizada para calcular el autovalor de menor magnitud de una matriz cuadrada. Es una variante del **método de la potencia**, que normalmente encuentra el autovalor de mayor magnitud. Para ello, se aprovecha la propiedad de la matriz inversa:

$$ A^{-1} v = \frac{1}{\lambda} v $$


## Algoritmo

El método de la potencia inversa sigue los siguientes pasos:

1. Elegir un vector inicial $x^{(0)}$ no nulo.

2. Para cada iteración $k$, resolver el sistema lineal:
   
   $$ A y^{(k)} = x^{(k-1)} $$
   
   donde $y^{(k)}$ es un **vector intermedio** que se obtiene resolviendo el sistema lineal. Este vector representa la solución del sistema en cada iteración y es crucial para aproximar el autovector de menor autovalor.

3. Normalizar $y^{(k)}$ para obtener un nuevo vector de iteración:
   
   $$ x^{(k)} = \frac{y^{(k)}}{\| y^{(k)} \|} $$
   
4. Aproximar el autovalor mediante el cociente de Rayleigh:
   
   $$ \lambda^{(k)} = \frac{(x^{(k)})^T A x^{(k)}}{(x^{(k)})^T x^{(k)}} $$
   
5. Repetir los pasos anteriores hasta que la diferencia entre valores sucesivos de $\lambda$ sea menor que un umbral de tolerancia.

---

*a)* Escriba una función llamada `potencia_inversa` que dada una matriz A encuentre su valor propio menos dominante (con una tolerancia de 1e-7 usando la norma Euclidiana) y su correspondiente autovector. Para obtener $y^{(k)}$ puede emplear `np.linalg.solve` recordando que $Ay = x$.

*b)* Usando la matriz A, encuentre el $\lambda_{min}$ y el correspondiente autovector. Examine la convergencia de su autovalor en función de las iteraciones, esto es, haga un gráfico de $\lambda(k)$

$$
\begin{bmatrix}
5 & 2 \\
2 & 2
\end{bmatrix}
$$

*c)* Compare su resultado con `np.linalg.eig`

In [57]:
#a
x_0=[1,1,1]
A=np.random.rand(5,5)


#4

**(10 puntos)**
Sea $A$ una matriz cuadrada de tamaño $n \times n$. Se sabe que el determinante de una matriz es igual al producto de sus autovalores:

$$
\det(A) = \prod_{i=1}^{n} \lambda_i
$$

donde $\lambda_1, \lambda_2, \dots, \lambda_n$ son los autovalores de $A$.

Usando una matriz aleatoria $A$ de $5\times 5$, demuestre esta propiedad. Puede usar las funciones de `np.linalg`.

In [93]:
#hacemos una matriz aleatoria
A=random.rand(5,5)
val_p=np.linalg.eig(A)[0]
val_p=np.copy(val_p)
val_p

array([ 2.1289441 +0.j        ,  0.68835897+0.j        ,
       -0.26272083+0.j        , -0.31082875+0.40685265j,
       -0.31082875-0.40685265j])

In [98]:
val_p[1]

(0.6883589735357643+0j)

In [67]:
A

array([[0.37327702, 0.58727307, 0.99722384, 0.91441152, 0.44182495],
       [0.11729259, 0.97713739, 0.61314422, 0.52854936, 0.89341958],
       [0.0602115 , 0.32785907, 0.45641371, 0.40465803, 0.97003601],
       [0.13764426, 0.81718969, 0.49994931, 0.98857081, 0.49347787],
       [0.70955655, 0.02898235, 0.7066502 , 0.04229238, 0.74452638]])

In [66]:
inverse=np.linalg.inv(A)
inverse

array([[-1.53419071, -0.47546026, -1.51882263,  2.20965997,  1.99525984],
       [-0.49866023,  1.67837532, -1.45885351,  0.15771317,  0.07808942],
       [ 2.39691117,  0.74167148,  0.27163013, -2.68695607, -0.8853637 ],
       [-0.19584925, -1.58337474,  0.67703067,  1.7636416 , -0.03480487],
       [-0.78230596, -0.22620411,  1.20800395,  0.33806707,  0.28085173]])

In [78]:
determinante=np.linalg.det(A)
determinante

0.12099455902292458

In [101]:
def factorial(val_p):
  factorial=1
  for i in range(len(val_p)):
    factorial*=val_p[i]
  return factorial

In [144]:
resultado=factorial(val_p)

In [146]:
print(f"{'correcto' if np.isclose(determinante, resultado) else 'no'}")

no
